# Validation et importation des métadonnées d'article <a class="anchor" id="top"></a>

Dans ce bloc-notes, vous reprenez là où vous vous êtes arrêté dans `01_Validating_and_Importing_User_Item_Interaction_Data.ipynb` pour créer un jeu de données opérationnel de métadonnées d'articles. Ce processus vous permettra de travailler avec des filtres et de prendre en charge ultérieurement les algorithmes `User Personalization` ou `HRNN-Metadata`.


Pour exécuter ce bloc-notes, vous devez avoir exécuté le bloc-notes précédent, `01_Validating_and_Importing_User_Item_Interaction_Data`, dans lequel vous avez créé un jeu de données et importé des données d'interaction dans Amazon Personalize. À la fin de ce bloc-notes, vous avez sauvegardé certaines des valeurs des variables, que vous devez maintenant charger dans ce bloc-notes.

In [ ]:
%store -r

## Préparer les métadonnées de votre article <a class="anchor" id="prepare"></a>
[Retour au début](#top)

Chargez ensuite les données et assurez-vous qu'elles sont en bon état, puis enregistrez-les dans un CSV où elles sont prêtes à être utilisées avec Amazon Personalize.

Pour commencer, importez une collection de bibliothèques Python couramment utilisées en science des données.

In [ ]:
import time
from time import sleep
import json
from datetime import datetime
import boto3
import pandas as pd

Ensuite, ouvrez le fichier de données et examinez les premières lignes.

In [ ]:
original_data = pd.read_csv(dataset_dir + '/movies.csv')
original_data.head(5)

In [ ]:
original_data.describe()

Comme nous n'en savons pas plus sur le jeu de données, nous allons donc explorer un peu plus pour obtenir des informations brutes. Nous pouvons constater que les genres sont souvent regroupés, ce qui nous convient puisque Personalize prend en charge cette structure.

In [ ]:
original_data.info()

Vous pouvez donc voir qu'il y a un total d'entrées (+ de 62 000 pour le jeu complet, 9 742 pour le jeu limité) dans le jeu de données, avec 3 colonnes.

Il s'agit d'un jeu de données assez minimal comprenant uniquement l'identifiant du film, le titre et la liste des genres applicables à chaque entrée. Toutefois, des données supplémentaires sont disponibles dans le jeu de données MovieLens. Par exemple, le titre comprend l'année de sortie du film. Faisons-en une autre colonne de métadonnées

In [ ]:
original_data['year'] =original_data['title'].str.extract('.*\((.*)\).*',expand = False)
original_data.head(5)

Du point de vue des métadonnées d'un article, nous ne voulons inclure que les informations qui sont pertinentes pour l'entraînement d'un modèle et/ou le filtrage des résultats. Nous allons donc oublier le titre, en conservant les informations sur le genre.

In [ ]:
itemmetadata_df = original_data.copy()
itemmetadata_df = itemmetadata_df[['movieId', 'genres', 'year']]
itemmetadata_df.head()

Après avoir manipulé les données, vérifiez toujours si le format des données a changé.

In [ ]:
itemmetadata_df.dtypes

Amazon Personalize dispose d'une colonne par défaut pour `ITEM_ID` qui correspondra à notre `movieId`, et nous pouvons maintenant étoffer les informations en spécifiant `GENRE` également.

In [ ]:
itemmetadata_df.rename(columns = {'genres':'GENRE', 'movieId':'ITEM_ID', 'year':'YEAR'}, inplace = True) 

Voilà ! À ce stade, les données sont prêtes à être utilisées et il ne nous reste plus qu'à les enregistrer dans un fichier CSV.

In [ ]:
itemmetadata_filename = "item-meta.csv"
itemmetadata_df.to_csv((data_dir+"/"+itemmetadata_filename), index=False, float_format='%.0f')

In [ ]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

### Créer le jeu de données

Tout d'abord, définissez un schéma afin d'indiquer à Amazon Personalize le type de jeu de données que vous chargez. Plusieurs mots-clés réservés et obligatoires sont requis dans le schéma, en fonction du type de jeu de données. Des informations plus détaillées peuvent être retrouvées dans la [documentation](https://docs.aws.amazon.com/personalize/latest/dg/how-it-works-dataset-schema.html).

Ici, vous allez créer un schéma pour les données de métadonnées d’articles, qui nécessite les champs `ITEM_ID` et `GENRE`. Ils doivent être définis dans le schéma en respectant l'ordre dans lequel ils apparaissent dans le jeu de données.

In [ ]:
itemmetadata_schema = {
    "type": "record",
    "name": "Items",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "GENRE",
            "type": "string",
            "categorical": True
        },{
            "name": "YEAR",
            "type": "int",
        },
        
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "personalize-poc-movielens-item",
    schema = json.dumps(itemmetadata_schema)
)

itemmetadataschema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

Une fois le schéma créé, vous pouvez créer un jeu de données dans le groupe de jeux de données. Il est à noter que cette opération ne charge pas encore les données. Cela arrivera quelques étapes plus tard.

In [ ]:
dataset_type = "ITEMS"
create_dataset_response = personalize.create_dataset(
    name = "personalize-poc-movielens-items",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = itemmetadataschema_arn
)

items_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

### Charger les données vers S3

Maintenant que votre compartiment Amazon S3 a été créé, chargez le fichier CSV de nos données d'interaction utilisateur-article. 

In [ ]:
itemmetadata_file_path = data_dir + "/" + itemmetadata_filename
boto3.Session().resource('s3').Bucket(bucket_name).Object(itemmetadata_filename).upload_file(itemmetadata_file_path)
interactions_s3DataPath = "s3://"+bucket_name+"/"+itemmetadata_filename

## Importer les métadonnées de l'article <a class="anchor" id="import"></a>
[Retour au début](#top)

Précédemment, vous avez créé le groupe de jeux de données et le jeu de données pour héberger vos informations. Vous allez maintenant exécuter une tâche d'importation qui chargera les données du compartiment S3 dans le jeu de données Amazon Personalize. 

In [ ]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-poc-item-import1",
    datasetArn = items_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, itemmetadata_filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

Pour pouvoir utiliser le jeu de données, la tâche d'importation doit être active. Exécutez la cellule ci-dessous et attendez qu'elle affiche le statut ACTIF. Il vérifie l'état de la tâche d'importation toutes les secondes, jusqu'à un maximum de 6 heures.

L'importation des données peut prendre un certain temps, en fonction de la taille du jeu de données. Dans cet atelier, l'importation des données devrait durer environ 15 minutes.

In [ ]:
%%time

max_time = time.time() + 6*60*60 # 6 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

Une fois cette importation terminée, vous pouvez activer le filtrage de vos recommandations ainsi que le support `HRNN-Metadata`. Exécutez la cellule ci-dessous avant de passer au stockage de quelques valeurs à utiliser dans les prochains blocs-notes. Après avoir rempli cette cellule, ouvrez le bloc-notes `03_Creating_and_Evaluating_Solutions.ipynb` pour continuer.

In [ ]:
%store items_dataset_arn
%store itemmetadataschema_arn